In [ ]:
import pandas as pd

data_path= 'C:\\Users\\c.hernandezperez\\Desktop\\drop\\reports\\PRSA_Ireland-Pensions-sit_2019-01-29 16-24.html'
#prueba
df = pd.read_html(data_path)
quit()
#begin_test_condition = df[0]['Details'].iloc[0]
#print(begin_test_condition)
display(df)
# display(df)
# print(type(df))

In [ ]:
#display(df[0]['Status'].iloc[-1])# print(len(df))
print(type(df[0]))
display(df[0])



# print(df[0].Status.value_counts()30
# 'cancel' in df[30]['Status'].tolist()

In [7]:
from bs4 import BeautifulSoup

'''
Workflow
1- Sumar pasos
2 - Checkear si el ultimo es un cancel
   2.1 - Si es un cancel, guardar la informacion en la primera fila del DF_N
   2.2 - Si no es un cancel, checkear si el test acabo
       2.2.1 - Si aun no acabo sumar los pasos a los que habia antes
       2.2.2 - Guardar los datos en el DF y comenzar una fila nueva
       
       
TODO

Hacer cambios para poder trabajar con varios workflows

'''
def extract_info(df_path):
    print(df_path)
    try:
        df = pd.read_html(df_path)
    except:
        data_path= 'C:\\Users\\c.hernandezperez\\Desktop\\drop\\reports\\PRSA_Ireland-Pensions-sit_2019-01-29 12-54.html'
        soup = BeautifulSoup(open(data_path), "html.parser")
        #prueba
        df = pd.read_html(str(soup.find_all('table')))
    # On this list we will be adding our dictionaries to improve speed
    rows_list = []
    steps = 0
    start_time=0
    end_time=0
    try:
        print(df[0]['Details'].iloc[0])
    except:
        import pdb;pdb.set_trace()
        pass
    
    # This dictionary will be the one filling the rows_list
    dict_test = {}
    df_extracted = pd.DataFrame(columns=['Steps','Status','Start Time', 'End Time'])
    # Boolean to check if we are still on a test case
    in_test = False


    import pdb;pdb.set_trace()
    for i in range(len(df)-1):
        #Starting a test case
     #   import pdb;pdb.set_trace()
        # This if statement is the weakest part, should modify for multi work-flows
        
        if len(df[i])==2 and df[i]['Timestamp'].iloc[0]==df[i]['Timestamp'].iloc[-1] and in_test==False:
            #import pdb; pdb.set_trace()
            pass
        else:
            # This if statement is the weakest part, should modify for multi work-flows
            if "Loaded the URL" in df[i]['Details'].iloc[0] and df[i]['Status'].sum()!=1:
                print('first time it enters in a test')
               # import pdb;pdb.set_trace()
                 # If we are on one work-flow this will make sure to save the previous test case
                # before getting on the next one
                if in_test:
                    #import pdb;pdb.set_trace()
                    status = 'pass'
                    #import pdb; pdb.set_trace()
                    # Add a new row
                    dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                    rows_list.append(dict_test)
                    dict_test = {}
                    in_test = False


              #  import pdb;pdb.set_trace()
                # If we have a fail on the first mayor step, it will stop here and save the test in the list
                if ('cancel' or 'warning') in df[i]['Status'].tolist():
                    # Values of the test
                    steps = df[i].Status.value_counts()['info_outline']
                    start_time = df[i]['Timestamp'].iloc[0]
                    end_time   = df[i]['Timestamp'].iloc[-1]
                    status = 'fail'
                
                    # Add a new row
                    dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                    rows_list.append(dict_test)
                    dict_test = {}
                    in_test = False




                # Recopilar pasos
                # The first time it enters a test
                else:
                    start_time = df[i]['Timestamp'].iloc[0]
                    end_time = df[i]['Timestamp'].iloc[-1]
                    steps = df[i].Status.value_counts()['info_outline']
                    in_test = True

                    # If it is the same 
                    if i==(len(df)-2):
                        status = 'pass'
                        # Add a new row
                        dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                        rows_list.append(dict_test)
                        dict_test = {}


            #We will enter here if there is continuation on a text or if the test was fatal
            else:
                if df[i].Status.value_counts().sum()==1 and ('redo' in df[i]['Status'].tolist() or 'cancel' or 'warning' in df[i]['Status'].tolist()):
                   # import pdb;pdb.set_trace()
                    
                    if in_test:
                        status = 'pass'
                        # Add a new row
                        dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                        rows_list.append(dict_test)
                        dict_test = {}
                        in_test = False
                    
                    
                    # In this case we do not need the "tolist()" method
                    if ('cancel' or 'warning') in df[i].Status.value_counts():
                        status = 'fatal'
                    else:
                        status = 'skip'
                    steps = 1
                    start_time = df[i]['Timestamp'].iloc[0]
                    end_time   = df[i]['Timestamp'].iloc[0]

                    # Add a new row
                    dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                    rows_list.append(dict_test)
                    dict_test = {}
                else:
                    # Continuing the test case but it is a failed test
                    if ('cancel' or 'warning') in df[i]['Status'].tolist():
                        #print('seems to be working')
                      #  import pdb;pdb.set_trace()
                        try:
                            steps += df[i].Status.value_counts()['info_outline']
                        except:
                            pass
                        end_time = df[i]['Timestamp'].iloc[-1]
                        status = 'fail'

                        # Add a new row
                        dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                        rows_list.append(dict_test)
                        dict_test = {}
                        in_test = False

                    # Continuing the test case
                    else:
                       # print('continue on the test testing')
                        #import pdb;pdb.set_trace()
                        try:
                            steps += df[i].Status.value_counts()['info_outline']
                            end_time = df[i]['Timestamp'].iloc[-1]
                        except:
                            pass
                          #  import pdb;pdb.set_trace()
                        in_test = True
                        if i==(len(df)-2):
                           # import pdb;pdb.set_trace()
                            status = 'pass'
                            # Add a new row
                            dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
                            rows_list.append(dict_test)
                            dict_test = {}

                        # Añadir al dataframe
    df_extracted = pd.DataFrame(rows_list)
    
    soup = BeautifulSoup(open(df_path), "html.parser")
    test_names = []

    for test in soup.find_all('span', {'class' : 'test-name'}):
        print(test.text)
        test_names.append(test.text)
    try:
        df_extracted['test_names'] = test_names
    except:
        
        # Add a new row
        status = 'pass'
        dict_test = {'Steps' : steps, 'Status' : status, 'Start Time' : start_time, 'End Time' : end_time}
        rows_list.append(dict_test)
        df_extracted = pd.DataFrame(rows_list)
        import pdb;pdb.set_trace()
        df_extracted['test_names'] = test_names
        
    return df_extracted
    # df_extraced = pd.concat([df_extracted, df_middle])

# display(df_middle)

In [8]:
import os
import pandas as pd

# Read all the file names of a given directory
data_path = 'C:\\Users\\c.hernandezperez\\Desktop\\drop\\reports\\' # Cambiar esto
dirs = os.listdir(data_path)
# Open all the excel files and concatenate the data into a single variable
df = pd.DataFrame()
for name in dirs:
    extracted = extract_info(data_path+'PRSA_Ireland-Pensions-sit_2019-01-29 12-54.html')
    display(extracted)

    df = pd.concat([df, extracted], axis=0)
df.to_excel(data_path+'prueba_2.xlsx',index=False)

C:\Users\c.hernandezperez\Desktop\drop\reports\PRSA_Ireland-Pensions-sit_2019-01-29 12-54.html
'TC_081 RETRYing - FAILED at 'Fill Plan Details'
> <ipython-input-7-353a2df50ce9>(46)extract_info()
-> for i in range(len(df)-1):
(Pdb) df[0]
  Status    Timestamp                                           Details
0   redo  12:57:09 PM  'TC_081 RETRYing - FAILED at 'Fill Plan Details'
(Pdb) exit()


BdbQuit: 

In [12]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(open('C:\\Users\\c.hernandezperez\\Desktop\\drop\\reports\\342.html'), "html.parser")


In [19]:
test_names = [] 
for test in soup.find_all('span', {'class' : 'test-name'}):
    print(test.text)
    test_names.append(test.text)
print(test_names)

BT_AustraliaNewSubmission
BT_ChinaNewSubmissionRenewal
BT_HongKongNewSubmissionCopySubmission
BT_IndonesiaNewSubmission
BT_SingaporeNewSubmissionCancellation
BT_SpainNewSubmissionMultiversion
BT_SwitzerlandNewSubmissionChangePolicyEffDate
BT_AustraliaMta
['BT_AustraliaNewSubmission', 'BT_ChinaNewSubmissionRenewal', 'BT_HongKongNewSubmissionCopySubmission', 'BT_IndonesiaNewSubmission', 'BT_SingaporeNewSubmissionCancellation', 'BT_SpainNewSubmissionMultiversion', 'BT_SwitzerlandNewSubmissionChangePolicyEffDate', 'BT_AustraliaMta']
